# Using database with python
- Use of an memory database like SQL Lite
- Create Database schema and relations
- Create Sql commands CRUD opération
- Read raw data from a file, process it and save record in table

In [48]:
# import of the sqllite3 module
import sqlite3
import re

# Creation of an instance conn for connexion
conn = sqlite3.connect("mbox.sqlite")
# creation of a cursor instance
cur = conn.cursor()

# Creation of the table Count if not exists
cur.execute('''DROP TABLE IF EXISTS Counts''') 

cur.execute('''
CREATE TABLE IF NOT EXISTS Counts(org TEXT, count INTEGER)
''')

# mail box file procession
fname = input("Enter filename:")
if len(fname) < 1:
    fname = 'mbox.txt'
with open(fname) as f:
    for line in f:
        # skipping unwanted lines
        if not line.startswith("From:"): continue
        line.rstrip() # deletion of whitespaces and tabs
        # use of  regex to check wheter a line startwith
        # From:
        if re.search('^From:', line):
            #print(line)
            # splitting of the line in order to extract the email
            #pieces = line.split() 
            #email = pieces[1]
            matches = re.findall('^From: (\S.+@.\S+)', line)
            email = matches[0]
            email = email.split("@")
            org = email[1]
            cur.execute('SELECT count FROM Counts WHERE org = ?', (org,))
            row = cur.fetchone()
            if row is None:
                cur.execute(''' INSERT INTO Counts(org, count) 
                                VALUES (?, 1)''', (org, ))
            else:
                cur.execute('UPDATE Counts SET count = count + 1 WHERE org = ?', (org, ))
            conn.commit()
            
# retrieving all current records in the relation Count      
sqlstr = 'SELECT org, count FROM Counts ORDER BY count DESC LIMIT 10'
    
for row in cur.execute(sqlstr):
        print(str(row[0]), row[1])
cur.close()

Enter filename:
iupui.edu 536
umich.edu 491
indiana.edu 178
caret.cam.ac.uk 157
vt.edu 110
uct.ac.za 96
media.berkeley.edu 56
ufp.pt 28
gmail.com 25
et.gatech.edu 17
